In [1]:
import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + '/../src')

In [2]:
import wandb
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pickle
from collections import deque
from recurrent.parameters import AllLogs
import jax 
jax.config.update("jax_platform_name", "cpu")


In [3]:
api = wandb.Api()
download_dir = "downloaded_artifacts"
entity = "wlp9800-new-york-university" 
project_name = "oho_exps"
group_name = "mlr_search-1_aa9c06652fb34624bebe972b1fe7292f"
group_name = "time_test_oho_d1efc05e0903463ca4e95a52714389a0"

os.makedirs(download_dir, exist_ok=True)

# Query runs in the specified project and group
runs = api.runs(
    path=f"{entity}/{project_name}" if entity else project_name,
    filters={"group": group_name}
)

for run in runs:
    print(f"Processing run: {run.id}")
    
    # Get all artifacts logged by this run
    artifact = run.use_artifact(f'{entity}/{project_name}/logs_{run.id}:v0', type='logs')
    print(f"Downloading artifact: {artifact.name}")
    # Construct a unique download path to avoid overwriting
    artifact_dir = os.path.join(download_dir, artifact.name)
    try:
        # Download the artifact
        artifact.download(root=artifact_dir)
        print(f"Downloaded {artifact.name} to {artifact_dir}")
    except Exception as e:
        print(f"Error downloading artifact {artifact.name}: {str(e)}")

Processing run: 83hjf1k8


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb:   1 of 1 files downloaded.  


Downloaded env_83hjf1k8:v0 to downloaded_artifacts/83hjf1k8/env_83hjf1k8:v0


wandb:   1 of 1 files downloaded.  


Downloaded env_83hjf1k8:v1 to downloaded_artifacts/83hjf1k8/env_83hjf1k8:v1


...

Downloaded logs_vuae6fly:v0 to downloaded_artifacts/vuae6fly/logs_vuae6fly:v0


wandb:   1 of 1 files downloaded.  


Downloaded env_vuae6fly:v2 to downloaded_artifacts/vuae6fly/env_vuae6fly:v2


wandb:   1 of 1 files downloaded.  


Downloaded run-vuae6fly-history:v0 to downloaded_artifacts/vuae6fly/run-vuae6fly-history:v0
